In [1]:
import os
os.chdir("../")
os.chdir("FinalDataset")

In [2]:
os.getcwd()

'/Volumes/Rohith/url_classification_dl/FinalDataset'

In [3]:
import pandas as pd
import tensorflow as ts
import numpy as np

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


In [4]:
data = pd.read_csv("URL_features.csv")

In [8]:
y.describe()

count    165366.000000
mean          1.194224
std           1.067849
min           0.000000
25%           1.000000
50%           1.000000
75%           1.000000
max           4.000000
Name: File, dtype: float64

In [5]:
data.drop(columns='Unnamed: 0',inplace=True)
data.replace('Benign_list_big_final',0,inplace=True)
data.replace('DefacementSitesURLFiltered',1,inplace=True)
data.replace('Malware_dataset',2,inplace=True)
data.replace('phishing_dataset',3,inplace=True)
data.replace('spam_dataset',4,inplace=True)
data.replace(True,1,inplace = True)
data.replace(False,0,inplace = True)
y = data["File"]
data.drop(columns = "ext",inplace = True)

In [6]:
data = data.drop(columns = "File")

In [7]:
y.head()

0    1
1    1
2    1
3    1
4    1
Name: File, dtype: int64

In [ ]:
data.head()

In [9]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Masking, Embedding

model = Sequential()


# Recurrent layer
#model.add(LSTM(64, return_sequences=False, 
 #              dropout=0.1, recurrent_dropout=0.1))

# Fully connected layer
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))

# Dropout for regularization
model.add(Dropout(0.5))

# Output layer
model.add(Dense(1, activation='softmax'))

# Compile the model
model.compile(
    optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy','mae'])

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, y, test_size=0.25, random_state=42)

In [11]:
model.fit(X_train,y_train,epochs = 10, batch_size = 128)

Epoch 1/10
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: invalid syntax (tmpee4dcqkk.py, line 48)
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: invalid syntax (tmpee4dcqkk.py, line 48)
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
969/969 [==============================] - 18s 963us/step - loss: 0.0000e+00 - accuracy: 0.5814 - mae: 0.6245
Epoch 2/10
969/969 [==============================] - 1s 926us/step - loss: 0.0000e+00 - accuracy: 0.5828 - mae: 0.6226
Epoch 3/10
969/969 [==============================] - 1s 958us/step - loss: 0.0000e+00 - accuracy: 0.5842 - mae: 0.6195
Epoch 4/10
969/969 [===============

In [12]:
y_pred = model.predict(X_test)

In [13]:
from sklearn.metrics import classification_report

In [15]:
target_names = ['DefacementSitesURLFiltered','phishing_dataset','Malware_dataset','spam_dataset','Benign_list_big_final']
print(classification_report(y_test, y_pred))#, target_names=target_names))

SyntaxError: unexpected EOF while parsing (<ipython-input-15-838286b67074>, line 2)